# Modelo Baseline

* Partiendo de las tablas tratadas.

In [78]:
import pandas as pd
import numpy as np
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from math import pi
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from ydata_profiling import ProfileReport
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score


%matplotlib inline
# Map Stuff
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
# pip install ydata-profiling
# pip install -U googlemaps
# pip install geopy

In [3]:
mypth = "/Users/cesarrojasflores/Documents/DataMining/moroccotopo/Data/"

In [4]:
w_outliers = pd.read_csv(mypth+"variables_con_atípicos.csv")
wo_outliers = pd.read_csv(mypth+"variables_sin_atípicos.csv")
categoric = pd.read_csv(mypth+"variables_categoricas_airbnb.csv")
categoric_excel = pd.read_excel(mypth+"variables_categoricas_airbnb.xlsx")

In [8]:
catvrs = categoric_excel.columns.to_list()
numvrs = list(set(w_outliers.columns.to_list()) - set(catvrs))

In [9]:
"""
# Verificando que los headers sean iguales
catvrs1 = categoric.columns.to_list()
if catvrs == catvrs1:
    print("Las listas son iguales")
else:
    print("Las listas son diferentes")

print(len(catvrs))
print(len(catvrs1))
"""

'\n# Verificando que los headers sean iguales\ncatvrs1 = categoric.columns.to_list()\nif catvrs == catvrs1:\n    print("Las listas son iguales")\nelse:\n    print("Las listas son diferentes")\n\nprint(len(catvrs))\nprint(len(catvrs1))\n'

In [19]:
print(w_outliers.shape)
print(wo_outliers.shape)

(23338, 58)
(19789, 58)


In [21]:
# Tamaño de particiones (e.g., 70% train, 15% validation, 15% test)
train_size = 0.7
validation_size = 0.15
test_size = 0.15

# Dividir en train y (validation + test)
train_data, temp_data = train_test_split(w_outliers, test_size=(1 - train_size), random_state=42)

# Dividir temp_data en validation y test
validation_data, test_data = train_test_split(temp_data, test_size=test_size / (test_size + validation_size), random_state=42)


In [41]:
train_data.shape

(16336, 61)

In [23]:
train_data["log_price"]= np.log(train_data.price)
train_data['boxcox_host_total_listings_count'], fitted_lambda = stats.boxcox(train_data['host_total_listings_count'])
train_data["log_host_total_listings_count"]= np.log(train_data.host_total_listings_count)

In [27]:
profile = ProfileReport(train_data[numvrs+["log_price","log_host_total_listings_count","boxcox_host_total_listings_count"]].copy(), minimal=True)
profile.to_file("Arbnb_EDA_num.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
Q1 = train_data['boxcox_host_total_listings_count'].quantile(0.25)
Q3 = train_data['boxcox_host_total_listings_count'].quantile(0.75)
IQR = Q3 - Q1
wo_outliers_train_data = train_data[~((train_data['boxcox_host_total_listings_count'] < (Q1 - 1.5 * IQR)) | 
                                         (train_data['boxcox_host_total_listings_count'] > (Q3 + 1.5 * IQR)))]

In [35]:
Q1 = wo_outliers_train_data['log_price'].quantile(0.25)
Q3 = wo_outliers_train_data['log_price'].quantile(0.75)
IQR = Q3 - Q1
wo_outliers_train_data = wo_outliers_train_data[~((wo_outliers_train_data['log_price'] < (Q1 - 1.5 * IQR)) | 
                                         (wo_outliers_train_data['log_price'] > (Q3 + 1.5 * IQR)))]

In [37]:
wo_outliers_train_data.shape

(16046, 61)

In [43]:
final_nums = [element for element in numvrs if element != "price" and element != "host_total_listings_count" and element != "Unnamed: 0"] + ["log_price","boxcox_host_total_listings_count"]
num_fin = wo_outliers_train_data[final_nums]

In [45]:
selector = VarianceThreshold(threshold=0.1)  # Elimina características con varianza menor a 0.1
df_reduced_num = selector.fit_transform(num_fin)
df_reduced_num = pd.DataFrame(df_reduced_num, columns=num_fin.columns[selector.get_support()])

In [47]:
# Los ratios de aceptación fueron eliminados por baja variación se forzó mantener la latidu y la longitud
lreducednum = df_reduced_num.columns.to_list()
list(set(wo_outliers_train_data[final_nums].columns.to_list()) - set(lreducednum))

['host_acceptance_rate', 'longitude', 'host_response_rate', 'latitude']

In [49]:
final_nums = list(set(wo_outliers_train_data[final_nums].columns.to_list()) - set(["host_acceptance_rate","host_response_rate"]))
wo_outliers_train_data[final_nums]

,calculated_host_listings_count_shared_rooms,accommodates,calculated_host_listings_count_entire_homes,longitude,beds,calculated_host_listings_count_private_rooms,log_price,boxcox_host_total_listings_count,latitude,bedrooms
7861,0,4,8,-99.163020,2.0,0,6.963190,1.732459,19.432250,2.0
14494,0,4,1,-99.160202,2.0,0,7.568379,0.000000,19.421836,2.0
19524,0,2,7,-99.166095,1.0,0,7.242082,1.659754,19.425170,1.0
11165,0,4,0,-99.134930,2.0,4,7.154615,1.988607,19.381810,2.0
12509,0,4,12,-99.176576,2.0,4,7.044905,2.503958,19.443067,2.0
...,...,...,...,...,...,...,...,...,...,...
11964,0,4,2,-99.183102,2.0,0,7.317876,0.973040,19.440236,2.0
21575,0,2,0,-99.120110,1.0,7,5.683580,1.732459,19.485270,1.0
5390,0,4,1,-99.134370,2.0,0,6.431331,0.000000,19.452840,1.0
860,0,1,0,-99.198350,1.0,7,6.396930,1.574956,19.341480,1.0


In [53]:
catvrsf = [element for element in catvrs if element != "id"  and element != "Unnamed: 0"]

In [55]:
profile = ProfileReport(wo_outliers_train_data[catvrsf].copy(), minimal=True)
profile.to_file("Arbnb_EDA_cat.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
# Eliminando host_identity_verified y host_has_profile_pic debido a que son constantes
catvrsf = list(set(catvrsf) - set(["host_identity_verified","host_has_profile_pic"]))

In [59]:
selector = VarianceThreshold(threshold=0.2)  # Elimina características con varianza menor a 0.9
df_reduced_cat = selector.fit_transform(wo_outliers_train_data[catvrsf])
df_reduced_cat = pd.DataFrame(df_reduced_cat, columns=wo_outliers_train_data[catvrsf].columns[selector.get_support()])

In [61]:
final_cats = list(set(wo_outliers_train_data[catvrsf].columns.to_list()) - set(df_reduced_cat.columns.to_list()))
print(len(final_cats))
print(len(catvrsf))

31
43


In [63]:
presel = final_cats+final_nums

In [65]:

df = wo_outliers_train_data[presel].copy()

# Crear transformaciones cuadráticas y cúbicas de cada variable
for col in df.columns.drop('log_price'):
    df[f'{col}_squared'] = df[col] ** 2
    df[f'{col}_cubed'] = df[col] ** 3

# Separar características y variable objetivo
X = df.drop(columns=['log_price'])
y = df['log_price']

# Calcular la información mutua de cada variable con respecto a 'log_price'
mi_scores = mutual_info_regression(X, y)
mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mi_scores})

# Filtrar por las variables con la información mutua más alta
mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)

# Eliminar variables correlacionadas
# Calculamos la matriz de correlación y aplicamos un umbral para seleccionar las no redundantes
correlation_matrix = X.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Umbral de correlación alta (ajustable según tus necesidades, e.g., 0.8)
threshold = 0.5
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

# Filtramos las variables menos correlacionadas y más informativas
X_filtered = X.drop(columns=to_drop)
mi_filtered_df = mi_df[~mi_df['Feature'].isin(to_drop)]

# Mostrar las variables seleccionadas y sus puntajes de información mutua
print("Variables seleccionadas después de eliminar las correlacionadas y su información mutua:")
print(mi_filtered_df)

# Si deseas mostrar el conjunto de datos filtrado
X_filtered['log_price'] = y  # Agregamos la variable objetivo para referencia
print("Conjunto de datos final con variables transformadas y filtradas:")
print(X_filtered.head())


Variables seleccionadas después de eliminar las correlacionadas y su información mutua:
                                        Feature  Mutual Information
33  calculated_host_listings_count_entire_homes            0.353042
32                                 accommodates            0.289169
38                                     latitude            0.242637
34                                    longitude            0.228261
2                         bathrooms_type_shared            0.123684
31  calculated_host_listings_count_shared_rooms            0.057749
0                         room_type_Shared_room            0.036599
30                      alcaldia_Miguel_Hidalgo            0.031787
13                                host_location            0.024569
12        host_response_time_within_a_few_hours            0.020161
4                        alcaldia_Benito_Juárez            0.015629
19                host_verifications_work_email            0.015204
29                          

In [68]:
# Información mutua mayor a 0.011
final_sel = mi_filtered_df.head(16).Feature.to_list()
wo_outliers_train_data[final_sel].corr()

,calculated_host_listings_count_entire_homes,accommodates,latitude,longitude,bathrooms_type_shared,calculated_host_listings_count_shared_rooms,room_type_Shared_room,alcaldia_Miguel_Hidalgo,host_location,host_response_time_within_a_few_hours
calculated_host_listings_count_entire_homes,1.000000,0.063682,0.100346,-0.065796,-0.121281,-0.033349,-0.040695,0.055238,-0.129780,-0.088689
accommodates,0.063682,1.000000,0.016084,0.012711,-0.259148,-0.067756,-0.059727,0.035712,-0.028849,-0.047093
latitude,0.100346,0.016084,1.000000,0.109558,-0.013980,0.093899,0.033138,0.269833,-0.044546,-0.048905
longitude,-0.065796,0.012711,0.109558,1.000000,0.048586,0.045571,0.055255,-0.336337,-0.031995,-0.008366
bathrooms_type_shared,-0.121281,-0.259148,-0.013980,0.048586,1.000000,0.203958,0.328016,-0.033004,0.044120,0.074672
calculated_host_listings_count_shared_rooms,-0.033349,-0.067756,0.093899,0.045571,0.203958,1.000000,0.429328,-0.029338,0.042568,0.208330
room_type_Shared_room,-0.040695,-0.059727,0.033138,0.055255,0.328016,0.429328,1.000000,-0.022727,0.004639,0.070355
alcaldia_Miguel_Hidalgo,0.055238,0.035712,0.269833,-0.336337,-0.033004,-0.029338,-0.022727,1.000000,0.002997,-0.017596
host_location,-0.129780,-0.028849,-0.044546,-0.031995,0.044120,0.042568,0.004639,0.002997,1.000000,0.030345
host_response_time_within_a_few_hours,-0.088689,-0.047093,-0.048905,-0.008366,0.074672,0.208330,0.070355,-0.017596,0.030345,1.000000


In [80]:


# Split the data into training, validation, and test sets (60%-20%-20%)
X_train = wo_outliers_train_data[final_sel]
y_train = wo_outliers_train_data["log_price"]
X_val = validation_data[final_sel]
X_test = test_data[final_sel]
y_val = np.log(validation_data["price"])
y_test = np.log(test_data["price"])

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Initialize and fit the Ridge regression model
# Use the validation set to tune alpha
best_alpha = 0.1  # Initial guess for alpha
best_val_score = float('-inf')
alphas = [0.01, 0.1, 0.5, 1.0]  # Example alphas for tuning

for alpha in alphas:
    ridge =Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    val_score = r2_score(y_val, ridge.predict(X_val_scaled))
    if val_score > best_val_score:
        best_val_score = val_score
        best_alpha = alpha

print(f"Best alpha based on validation set: {best_alpha}")

# Train the final model with the best alpha on the training set
final_model = Ridge(alpha=best_alpha)
final_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_train_pred = final_model.predict(X_train_scaled)
y_val_pred = final_model.predict(X_val_scaled)
y_test_pred = final_model.predict(X_test_scaled)

print("Training RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Validation RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print("Training R^2:", r2_score(y_train, y_train_pred))
print("Validation R^2:", r2_score(y_val, y_val_pred))
print("Test R^2:", r2_score(y_test, y_test_pred))

# Display the coefficients
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Ridge Coefficient': final_model.coef_})
print("\Ridge Coefficients:")
print(coefficients)


Best alpha based on validation set: 0.01
Training RMSE: 0.5155790592124856
Validation RMSE: 0.5440384032333262
Test RMSE: 0.5618139808279051
Training R^2: 0.4350426605625257
Validation R^2: 0.47439444372002804
Test R^2: 0.44942693516266763

Lasso Coefficients:
                                       Feature  Ridge Coefficient
0  calculated_host_listings_count_entire_homes           0.623473
1                                 accommodates           2.068846
2                                     latitude           0.574185
3                                    longitude          -1.476440
4                        bathrooms_type_shared          -0.508608
5  calculated_host_listings_count_shared_rooms          -0.734392
6                        room_type_Shared_room          -0.218010
7                      alcaldia_Miguel_Hidalgo           0.040079
8                                host_location          -0.004571
9        host_response_time_within_a_few_hours          -0.059058
